# Building Dynamic Models with the Subclassing API

Waar de `Sequential` en `Functional` API's **statisch** zijn (de architectuur ligt vast zodra je het model bouwt), biedt de **Subclassing API** volledige vrijheid voor **dynamisch gedrag**.

**Statisch vs. Dynamisch**

- **Statisch (Sequential/Functional):** Gemakkelijk op te slaan, te klonen en de structuur te visualiseren. Keras kan shapes en types vooraf controleren.
- **Dynamisch (Subclassing):** Maakt het gebruik van Python-logica mogelijk, zoals `for`loops, `if`statements (conditional branching) en variërende shapes.

## 1. Structuur van een Subclassed Model

Om deze API te gebruiken, maak je een class die overerft van `keras.Model`.

- **`__init__()`**: Hier maak je de lagen aan die je wilt gebruiken.
- **`call()`**: Hier definieer je de berekeningen (de "forward pass"). Dit is waar je eventuele dynamische logica kunt toevoegen.

**Voorbeeld: Wide & Deep Model**

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# 1. Data laden
housing = fetch_california_housing()

# 2. Splitsen in Train, Validatie en Test
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

# 3. De Wide en Deep subsets maken (DIT IS CRUCIAAL)
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

In [2]:
import keras

class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)  # Nodig voor modelnaam en andere basisfunctionaliteit

        # Lagen definiëren als attributen van de class
        self.norm_layer_wide = keras.layers.Normalization()
        self.norm_layer_deep = keras.layers.Normalization()
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_wide, input_deep = inputs

        # De dataflow (forward pass)
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)

        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)

        concat = keras.layers.concatenate([norm_wide, hidden2])

        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)

        return output, aux_output

# Model instantiëren
model = WideAndDeepModel(name="my_dynamic_model")

## **2. Gebruik: Adapt en Train**

Omdat de normalisatielagen nu "verstopt" zitten in het model-object, roep je ze aan via het model:

In [ ]:
# Adapt de lagen via het model object
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)

# Trainen gaat precies hetzelfde als bij de andere API's
model.compile(loss=["mse","mse"], loss_weights=[0.9, 0.1], optimizer="adam")
model.fit((X_train_wide, X_train_deep), (y_train, y_train), epochs=10)

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 323us/step - loss: 1.6028 - mse_loss: 2.8406
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 300us/step - loss: 0.6203 - mse_loss: 0.8929
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 291us/step - loss: 0.4864 - mse_loss: 0.6298
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - loss: 0.4275 - mse_loss: 0.5483
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 286us/step - loss: 0.4046 - mse_loss: 0.5125
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 283us/step - loss: 0.3850 - mse_loss: 0.4904
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 283us/step - loss: 0.3798 - mse_loss: 0.4720
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step - loss: 0.3833 - mse_loss: 0.4627
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step - loss: 0.3926 - mse_loss: 0.4679
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step - loss: 0.3747 - mse_loss: 0.4497
